# Competition Objective is to detect fraud in transactions; 

## Data


In this competition you are predicting the probability that an online transaction is fraudulent, as denoted by the binary target ```isFraud```.

The data is broken into two files **identity** and **transaction**, which are joined by ```TransactionID```. 

> Note: Not all transactions have corresponding identity information.

**Transaction variables**

- TransactionDT: timedelta from a given reference datetime (not an actual timestamp)
- TransactionAMT: transaction payment amount in USD
- ProductCD: product code, the product for each transaction
- card1 - card6: payment card information, such as card type, card category, issue bank, country, etc.
- addr: address
- dist: distance
- P_ and (R__) emaildomain: purchaser and recipient email domain
- C1-C14: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked.
- D1-D15: timedelta, such as days between previous transaction, etc.
- M1-M9: match, such as names on card and address, etc.
- Vxxx: Vesta engineered rich features, including ranking, counting, and other entity relations.

**Categorical Features - Transaction**

- ProductCD
- emaildomain
- card1 - card6
- addr1, addr2
- P_emaildomain
- R_emaildomain
- M1 - M9

**Categorical Features - Identity**

- DeviceType
- DeviceInfo
- id_12 - id_38

**The TransactionDT feature is a timedelta from a given reference datetime (not an actual timestamp).**


# 1. Importation and memory reduction
## 1.1. Importing necessary libraries

In [30]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy as sp
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

# Standard plotly imports
#import plotly.plotly as py
import plotly.graph_objs as go

import plotly.express as px
import plotly.tools as tls
from plotly.subplots import make_subplots

from plotly.offline import iplot, init_notebook_mode
#import cufflinks
#import cufflinks as cf
import plotly.figure_factory as ff


# Using plotly + cufflinks in offline mode
init_notebook_mode(connected=True)
#cufflinks.go_offline(connected=True)

# Preprocessing, modelling and evaluating
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold
from xgboost import XGBClassifier
import xgboost as xgb

## Hyperopt modules
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK, STATUS_RUNNING
from functools import partial

import os
import gc
import time

In [0]:
### Import data from google drive

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
link1 = 'https://drive.google.com/open?id=1-2YM1zPLC6L4e94c7UHtfm70-UhrJAx9'
link2 = 'https://drive.google.com/open?id=1-8b83OZ7qoWypGqquhyXXmbETyhjDq0G'

link5 = 'https://drive.google.com/open?id=1v5in81M0aYl-pX5u-8Tq4G8kwFp4CVez'
_, id1 = link1.split('=')
_, id2 = link2.split('=')

_, id5 = link5.split('=')

downloaded1 = drive.CreateFile({'id':id1}) 
downloaded1.GetContentFile('df_train.pkl')  
downloaded2 = drive.CreateFile({'id':id2}) 
downloaded2.GetContentFile('df_test.pkl')  

downloaded5 = drive.CreateFile({'id':id5}) 
downloaded5.GetContentFile('sample_submission.csv')

## 1.2. Importing train datasets

In [0]:
df_train = pd.read_pickle("df_train.pkl")
df_test = pd.read_pickle("df_test.pkl")
sample_submission = pd.read_csv('sample_submission.csv')

In [0]:
#df_train = df_train1[0:10000]

In [34]:
print(df_train.shape)
print(df_test.shape)

(590540, 430)
(506691, 429)


## 1.3. Memory reduction

In [0]:
def resumetable(df):
    n = df.shape[0]
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values  
    summary['Missing %'] = round(summary['Missing'] / n * 100,2)
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.loc[0].values
    summary['Second Value'] = df.loc[1].values
    summary['Third Value'] = df.loc[2].values


    return summary

## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True, object_tranform = False):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)   
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

def CalcOutliers(df_num): 

    # calculating mean and std of the array
    data_mean, data_std = np.mean(df_num), np.std(df_num)

    # seting the cut line to both higher and lower values
    # You can change this value
    cut = data_std * 3

    #Calculating the higher and lower cut values
    lower, upper = data_mean - cut, data_mean + cut

    # creating an array of lower, higher and total outlier values 
    outliers_lower = [x for x in df_num if x < lower]
    outliers_higher = [x for x in df_num if x > upper]
    outliers_total = [x for x in df_num if x < lower or x > upper]

    # array without outlier values
    outliers_removed = [x for x in df_num if x > lower and x < upper]
    print('Lower bounded value: {:.2f}'.format(lower))
    print('Upper bounded value: {:.2f}'.format(upper))
    print('Identified lowest outliers: %d' % len(outliers_lower)) # printing total number of values in lower cut of outliers
    print('Identified upper outliers: %d' % len(outliers_higher)) # printing total number of values in higher cut of outliers
    print('Total outlier observations: %d' % len(outliers_total)) # printing total number of values outliers of both sides
    print('Non-outlier observations: %d' % len(outliers_removed)) # printing total number of non outlier values
    print("Total percentual of Outliers: ", round((len(outliers_total) / len(outliers_removed) )*100, 4)) # Percentual of outliers in points
    
    return

To see the output of the Resume Table, click to see the output 

# 2. Preprocessing

In [36]:
df_train.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D8,D9,D10,...,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_19,id_20,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,_merge,dist1_na,dist2_na,_Weekdays,_Hours,_Days
0,2987000,0,86400,4.226562,W,13926,-inf,150,discover,142,credit,315,87,19.0,0.0,gmail.com,gmail.com,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,97.0,13.0,26.0,10.0,0.0,37.875,0.666504,13.0,...,0.0,0.0,0.0,0.0,70787.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,NotFound,52.0,-480.0,New,NotFound,166.0,542.0,144.0,Miss,New,NotFound,Android,Samsung,32.0,2220x1080,match_status:2,T,F,T,T,mobile,Others,both,False,True,5,0,2
1,2987001,0,86401,3.367188,W,2755,404,150,mastercard,102,credit,325,87,8.0,0.0,gmail.com,Maybe_P,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,97.0,8.0,0.0,10.0,0.0,37.875,0.666504,0.0,...,0.0,0.0,0.0,-5.0,98945.0,0.0,0.0,0.0,-5.0,0.0,0.0,100.0,NotFound,49.0,-300.0,New,NotFound,166.0,621.0,500.0,Miss,New,NotFound,iOS,Safari,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device,both,True,True,5,0,2
2,2987002,0,86469,4.078125,W,4663,490,150,visa,166,debit,330,87,287.0,0.0,outlook.com,Maybe_P,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,97.0,8.0,0.0,10.0,0.0,37.875,0.666504,0.0,...,0.0,0.0,0.0,-5.0,191631.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,NotFound,52.0,-300.0,Found,Found,121.0,410.0,142.0,Miss,Found,Found,Miss,Chrome,24.0,1920x1080,match_status:2,F,F,T,T,desktop,Windows,both,False,True,5,0,2
3,2987003,0,86499,3.912109,W,18132,567,150,mastercard,117,debit,476,87,8.0,0.0,yahoo.com,Maybe_P,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,0.0,37.875,0.666504,84.0,...,0.0,0.0,0.0,-5.0,221832.0,0.0,0.0,0.0,-6.0,0.0,0.0,100.0,NotFound,52.0,-300.0,New,NotFound,225.0,176.0,507.0,Miss,New,NotFound,Miss,Chrome,24.0,1920x1080,match_status:2,F,F,T,T,desktop,Windows,both,True,True,5,0,2
4,2987004,0,86506,3.912109,H,4497,514,150,mastercard,102,credit,420,87,8.0,0.0,gmail.com,Maybe_P,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,97.0,8.0,26.0,10.0,0.0,37.875,0.666504,15.0,...,0.0,0.0,0.0,0.0,7460.0,0.0,0.0,1.0,0.0,0.0,0.0,100.0,NotFound,52.0,-300.0,Found,Found,166.0,529.0,575.0,Miss,Found,Found,Mac,Chrome,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS,both,True,True,5,0,2


In [0]:
df_train.drop(['_merge'], axis='columns', inplace=True)
df_test.drop(['_merge'], axis='columns', inplace=True)

In [0]:
categorical_features = []
for col in df_train.columns.drop('isFraud') :
    if df_train[col].dtype == 'object' or df_test[col].dtype=='object':
        categorical_features.append(col)


In [39]:
categorical_resume = resumetable(df_train[categorical_features])
categorical_resume

Dataset Shape: (590540, 36)


,Name,dtypes,Missing,Missing %,Uniques,First Value,Second Value,Third Value
0,ProductCD,object,0,0.0,5,W,W,W
1,card1,object,0,0.0,13553,13926,2755,4663
2,card2,object,0,0.0,501,-inf,404,490
3,card3,object,0,0.0,115,150,150,150
4,card4,object,0,0.0,5,discover,mastercard,visa
5,card5,object,0,0.0,120,142,102,166
6,card6,object,0,0.0,5,credit,credit,debit
7,addr1,object,0,0.0,333,315,325,330
8,addr2,object,0,0.0,75,87,87,87
9,P_emaildomain,object,0,0.0,31,gmail.com,gmail.com,outlook.com


In [0]:
# Label Encoding
for f in df_train.columns:
    if  df_train[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(df_train[f].values) + list(df_test[f].values))
        df_train[f] = lbl.transform(list(df_train[f].values))
        df_test[f] = lbl.transform(list(df_test[f].values))  
df_train = df_train.reset_index()
df_test = df_test.reset_index()

In [41]:
"""
# Label Encoding
categorical_features_v2 = categorical_features.copy()
for f in categorical_features:
    if float(categorical_resume.loc[categorical_resume['Name']==f, 'Uniques']) > 8 :
        categorical_features_v2.remove(f)
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(df_train[f].values) + list(df_test[f].values))
        df_train[f] = lbl.transform(list(df_train[f].values))
        df_test[f] = lbl.transform(list(df_test[f].values))  
"""

"\n# Label Encoding\ncategorical_features_v2 = categorical_features.copy()\nfor f in categorical_features:\n    if float(categorical_resume.loc[categorical_resume['Name']==f, 'Uniques']) > 8 :\n        categorical_features_v2.remove(f)\n        lbl = preprocessing.LabelEncoder()\n        lbl.fit(list(df_train[f].values) + list(df_test[f].values))\n        df_train[f] = lbl.transform(list(df_train[f].values))\n        df_test[f] = lbl.transform(list(df_test[f].values))  \n"

In [42]:
"""categorical_resume_v2 = resumetable(df_train[categorical_features_v2])
categorical_resume_v2
"""

'categorical_resume_v2 = resumetable(df_train[categorical_features_v2])\ncategorical_resume_v2\n'

In [43]:
"""# One hot coding

merged_train_test = pd.concat([df_train[categorical_features_v2], df_test[categorical_features_v2]], axis =0, ignore_index=True)

for col in merged_train_test.columns :
    encoder = preprocessing.OneHotEncoder(sparse=False)
    a = pd.DataFrame(encoder.fit_transform(merged_train_test[[col]].astype(np.str)))
    a.columns = encoder.get_feature_names([col])
    merged_train_test.drop(col ,axis=1, inplace=True)
    merged_train_test= pd.concat([merged_train_test, a ], axis=1)
resumetable(merged_train_test)
"""

'# One hot coding\n\nmerged_train_test = pd.concat([df_train[categorical_features_v2], df_test[categorical_features_v2]], axis =0, ignore_index=True)\n\nfor col in merged_train_test.columns :\n    encoder = preprocessing.OneHotEncoder(sparse=False)\n    a = pd.DataFrame(encoder.fit_transform(merged_train_test[[col]].astype(np.str)))\n    a.columns = encoder.get_feature_names([col])\n    merged_train_test.drop(col ,axis=1, inplace=True)\n    merged_train_test= pd.concat([merged_train_test, a ], axis=1)\nresumetable(merged_train_test)\n'

In [44]:
"""
df_train.drop(categorical_features_v2, axis =1, inplace = True)
df_test.drop(categorical_features_v2, axis =1, inplace = True)
df_train = pd.concat([df_train, merged_train_test[0:len(df_train)]], axis = 1, ignore_index=True)
df_test = pd.concat([df_test, merged_train_test[len(df_train):].reset_index(drop= True)], axis = 1, ignore_index=True)

"""

'\ndf_train.drop(categorical_features_v2, axis =1, inplace = True)\ndf_test.drop(categorical_features_v2, axis =1, inplace = True)\ndf_train = pd.concat([df_train, merged_train_test[0:len(df_train)]], axis = 1, ignore_index=True)\ndf_test = pd.concat([df_test, merged_train_test[len(df_train):].reset_index(drop= True)], axis = 1, ignore_index=True)\n\n'

In [45]:
print(df_train.shape)
print(df_test.shape)

(590540, 430)
(506691, 429)


In [46]:
df_train.head()

,index,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D8,D9,...,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_19,id_20,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,dist1_na,dist2_na,_Weekdays,_Hours,_Days
0,0,2987000,0,86400,4.226562,4,12696,0,51,2,43,2,216,78,19.0,0.0,12,9,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,97.0,13.0,26.0,10.0,0.0,37.875,0.666504,...,0.0,0.0,0.0,0.0,0.0,70787.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,2,52.0,-480.0,2,2,166.0,542.0,144.0,1,2,2,0,8,32.0,268,3,1,0,1,1,1,1,False,True,5,0,2
1,1,2987001,0,86401,3.367188,4,1726,305,51,3,3,2,226,78,8.0,0.0,12,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,97.0,8.0,0.0,10.0,0.0,37.875,0.666504,...,0.0,0.0,0.0,0.0,-5.0,98945.0,0.0,0.0,0.0,-5.0,0.0,0.0,100.0,2,49.0,-300.0,2,2,166.0,621.0,500.0,1,2,2,6,7,32.0,80,2,1,0,0,1,1,4,True,True,5,0,2
2,2,2987002,0,86469,4.078125,4,3597,391,51,4,67,3,231,78,287.0,0.0,23,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,97.0,8.0,0.0,10.0,0.0,37.875,0.666504,...,0.0,0.0,0.0,0.0,-5.0,191631.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,2,52.0,-300.0,0,0,121.0,410.0,142.0,1,0,0,3,0,24.0,216,3,0,0,1,1,0,3,False,True,5,0,2
3,3,2987003,0,86499,3.912109,4,16830,468,51,3,18,3,377,78,8.0,0.0,29,0,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,0.0,37.875,0.666504,...,0.0,0.0,0.0,0.0,-5.0,221832.0,0.0,0.0,0.0,-6.0,0.0,0.0,100.0,2,52.0,-300.0,2,2,225.0,176.0,507.0,1,2,2,3,0,24.0,216,3,0,0,1,1,0,3,True,True,5,0,2
4,4,2987004,0,86506,3.912109,1,3434,415,51,3,3,2,321,78,8.0,0.0,12,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,97.0,8.0,26.0,10.0,0.0,37.875,0.666504,...,0.0,0.0,0.0,0.0,0.0,7460.0,0.0,0.0,1.0,0.0,0.0,0.0,100.0,2,52.0,-300.0,0,0,166.0,529.0,575.0,1,0,0,2,0,24.0,68,3,1,0,1,1,0,0,True,True,5,0,2


In [47]:
print(df_train.columns)

Index(['index', 'TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4',
       ...
       'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo', 'dist1_na',
       'dist2_na', '_Weekdays', '_Hours', '_Days'],
      dtype='object', length=430)


In [0]:
print(df_train['_merge'])

In [0]:
features = list(df_train)
features.remove('_merge')
features.remove('isFraud')
target = 'isFraud'

In [0]:
del df_train['_merge']

In [0]:
print(df_train['_merge'])

In [0]:
""""
clf = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=9,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    missing=-999,
    random_state=2019,
    tree_method='gpu_hist'  # THE MAGICAL PARAMETER
)

""""

In [0]:
""""
y_train = df_train['isFraud'].copy()

X_train = df_train.drop(['isFraud','_merge'], axis=1)
X_test = df_test.copy()
""""

In [0]:
""""

%time clf.fit(X_train, y_train)

""""

On réalise une ACP pour une réductiond des dimensions

In [0]:
df_test['isFraud'] = 'test'
df = pd.concat([df_train, df_test], axis=0, sort=False )
df = df.reset_index()
df = df.drop('index', axis=1)

def PCA_change(df, cols, n_components, prefix='PCA_', rand_seed=4):
    pca = PCA(n_components=n_components, random_state=rand_seed)

    principalComponents = pca.fit_transform(df[cols])

    principalDf = pd.DataFrame(principalComponents)

    df.drop(cols, axis=1, inplace=True)

    principalDf.rename(columns=lambda x: str(prefix)+str(x), inplace=True)

    df = pd.concat([df, principalDf], axis=1)
    
    return df



In [0]:


mas_v = df_train.columns[55:394]



In [0]:
from sklearn.preprocessing import minmax_scale
from sklearn.decomposition import PCA
# from sklearn.cluster import KMeans

for col in mas_v:
    df[col] = df[col].fillna((df[col].min() - 2))
    df[col] = (minmax_scale(df[col], feature_range=(0,1)))

    
df = PCA_change(df, mas_v, prefix='PCA_V_', n_components=30)

In [0]:
df.head()

In [0]:
df = reduce_mem_usage(df)

In [0]:


df_train, df_test = df[df['isFraud'] != 'test'], df[df['isFraud'] == 'test'].drop('isFraud', axis=1)



In [0]:


df_train.shape



In [0]:
df_train.head()

In [0]:
X_train = df_train.sort_values('TransactionDT').drop(['isFraud', 
                                                      'TransactionDT', 
                                                      #'Card_ID'
                                                     ],
                                                     axis=1)
y_train = df_train.sort_values('TransactionDT')['isFraud'].astype(bool)

X_test = df_test.sort_values('TransactionDT').drop(['TransactionDT',
                                                    #'Card_ID'
                                                   ], 
                                                   axis=1)
del df_train
df_test = df_test[["TransactionDT"]]

In [0]:
X_train.drop(['_merge'], axis='columns', inplace=True)
X_test.drop(['_merge'], axis='columns', inplace=True)


In [54]:
X_train.head()

,level_0,TransactionID,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D8,D9,D10,D11,...,id_36,id_37,id_38,DeviceType,DeviceInfo,dist1_na,dist2_na,_Weekdays,_Hours,_Days,PCA_V_0,PCA_V_1,PCA_V_2,PCA_V_3,PCA_V_4,PCA_V_5,PCA_V_6,PCA_V_7,PCA_V_8,PCA_V_9,PCA_V_10,PCA_V_11,PCA_V_12,PCA_V_13,PCA_V_14,PCA_V_15,PCA_V_16,PCA_V_17,PCA_V_18,PCA_V_19,PCA_V_20,PCA_V_21,PCA_V_22,PCA_V_23,PCA_V_24,PCA_V_25,PCA_V_26,PCA_V_27,PCA_V_28,PCA_V_29
0,0,2987000,4.226562,4,12696,0,51,2,43,2,216,78,19.0,0.0,12,9,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,97.0,13.0,26.0,10.0,0.0,37.875,0.666504,13.0,13.0,...,0,1,1,1,1,False,True,5,0,2,-0.153442,-0.234985,-0.087402,-0.005215,0.056976,-0.058868,-0.006348,-0.044739,0.006371,-0.037659,-0.006039,-0.039093,-0.008514,0.000394,-0.034973,-0.025589,0.099854,-0.000730,0.017654,0.044708,0.026871,0.004074,0.010635,0.021347,0.031464,-0.005894,0.006927,0.015488,-0.025665,-0.026749
1,1,2987001,3.367188,4,1726,305,51,3,3,2,226,78,8.0,0.0,12,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,97.0,8.0,0.0,10.0,0.0,37.875,0.666504,0.0,43.0,...,0,0,1,1,4,True,True,5,0,2,0.164795,-0.014389,0.027435,-0.333740,-0.109070,-0.023346,0.089172,-0.004261,-0.035889,-0.002920,-0.001410,0.046234,0.015686,-0.017471,-0.005611,-0.001104,0.020615,-0.007217,-0.007416,0.001747,-0.002581,0.002285,0.005829,0.009186,-0.016830,-0.004436,-0.000340,-0.001576,0.000975,-0.008194
2,2,2987002,4.078125,4,3597,391,51,4,67,3,231,78,287.0,0.0,23,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,97.0,8.0,0.0,10.0,0.0,37.875,0.666504,0.0,315.0,...,0,1,1,0,3,False,True,5,0,2,-0.145264,-0.236206,-0.085815,0.016129,-0.046417,-0.005280,-0.014015,0.006805,-0.028595,-0.035797,-0.005688,-0.007168,0.001621,-0.015465,-0.002512,-0.001182,0.003117,-0.002216,-0.005203,-0.003283,0.002920,0.005592,0.003241,0.010689,0.003262,0.001277,-0.005783,-0.000842,0.010750,-0.008667
3,3,2987003,3.912109,4,16830,468,51,3,18,3,377,78,8.0,0.0,29,0,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,0.0,37.875,0.666504,84.0,43.0,...,0,1,1,0,3,True,True,5,0,2,-0.148193,-0.243164,-0.090271,0.008652,-0.012314,0.077759,0.029861,-0.005665,0.002073,0.014030,-0.018631,0.000334,-0.088928,0.083984,0.106689,-0.060852,-0.007301,-0.020691,-0.010780,-0.017517,0.005154,0.033661,-0.007050,-0.012489,0.019699,0.033783,0.057556,-0.001304,0.006775,-0.007515
4,4,2987004,3.912109,1,3434,415,51,3,3,2,321,78,8.0,0.0,12,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,97.0,8.0,26.0,10.0,0.0,37.875,0.666504,15.0,43.0,...,0,1,1,0,0,True,True,5,0,2,-0.099121,-0.598633,1.275391,0.289795,0.001773,-0.058624,0.070374,0.016800,-0.019806,-0.014893,-0.015045,-0.012459,0.006882,-0.006195,0.003134,-0.030060,-0.005051,0.016800,0.009567,-0.048279,0.014519,-0.115356,-0.034576,0.003149,0.053558,0.000390,0.036530,0.019241,-0.062347,0.055756


In [0]:
from sklearn.model_selection import KFold,TimeSeriesSplit
from sklearn.metrics import roc_auc_score
from xgboost import plot_importance
from sklearn.metrics import make_scorer

import time
def objective(params):
    time1 = time.time()
    params = {
        'max_depth': int(params['max_depth']),
        'gamma': "{:.3f}".format(params['gamma']),
        'subsample': "{:.2f}".format(params['subsample']),
        'reg_alpha': "{:.3f}".format(params['reg_alpha']),
        'reg_lambda': "{:.3f}".format(params['reg_lambda']),
        'learning_rate': "{:.3f}".format(params['learning_rate']),
        'num_leaves': '{:.3f}'.format(params['num_leaves']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
        'min_child_samples': '{:.3f}'.format(params['min_child_samples']),
        'feature_fraction': '{:.3f}'.format(params['feature_fraction']),
        'bagging_fraction': '{:.3f}'.format(params['bagging_fraction'])
    }

    print("\n############## New Run ################")
    print(f"params = {params}")
    FOLDS = 7
    count=1
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)

    tss = TimeSeriesSplit(n_splits=FOLDS)
    y_preds = np.zeros(sample_submission.shape[0])
    y_oof = np.zeros(X_train.shape[0])
    score_mean = 0
    for tr_idx, val_idx in tss.split(X_train, y_train):
        clf = xgb.XGBClassifier(
            n_estimators=600, random_state=4, verbose=True, 
            tree_method='gpu_hist', 
            **params
        )

        X_tr, X_vl = X_train.iloc[tr_idx, :], X_train.iloc[val_idx, :]
        y_tr, y_vl = y_train.iloc[tr_idx], y_train.iloc[val_idx]
        
        clf.fit(X_tr, y_tr)
        #y_pred_train = clf.predict_proba(X_vl)[:,1]
        #print(y_pred_train)
        score = make_scorer(roc_auc_score, needs_proba=True)(clf, X_vl, y_vl)
        # plt.show()
        score_mean += score
        print(f'{count} CV - score: {round(score, 4)}')
        count += 1
    time2 = time.time() - time1
    print(f"Total Time Run: {round(time2 / 60,2)}")
    gc.collect()
    print(f'Mean ROC_AUC: {score_mean / FOLDS}')
    del X_tr, X_vl, y_tr, y_vl, clf, score
    return -(score_mean / FOLDS)


space = {
    # The maximum depth of a tree, same as GBM.
    # Used to control over-fitting as higher depth will allow model 
    # to learn relations very specific to a particular sample.
    # Should be tuned using CV.
    # Typical values: 3-10
    'max_depth': hp.quniform('max_depth', 7, 23, 1),
    
    # reg_alpha: L1 regularization term. L1 regularization encourages sparsity 
    # (meaning pulling weights to 0). It can be more useful when the objective
    # is logistic regression since you might need help with feature selection.
    'reg_alpha':  hp.uniform('reg_alpha', 0.01, 0.4),
    
    # reg_lambda: L2 regularization term. L2 encourages smaller weights, this
    # approach can be more useful in tree-models where zeroing 
    # features might not make much sense.
    'reg_lambda': hp.uniform('reg_lambda', 0.01, .4),
    
    # eta: Analogous to learning rate in GBM
    # Makes the model more robust by shrinking the weights on each step
    # Typical final values to be used: 0.01-0.2
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    
    # colsample_bytree: Similar to max_features in GBM. Denotes the 
    # fraction of columns to be randomly samples for each tree.
    # Typical values: 0.5-1
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, .9),
    
    # A node is split only when the resulting split gives a positive
    # reduction in the loss function. Gamma specifies the 
    # minimum loss reduction required to make a split.
    # Makes the algorithm conservative. The values can vary depending on the loss function and should be tuned.
    'gamma': hp.uniform('gamma', 0.01, .7),
    
    # more increases accuracy, but may lead to overfitting.
    # num_leaves: the number of leaf nodes to use. Having a large number 
    # of leaves will improve accuracy, but will also lead to overfitting.
    'num_leaves': hp.choice('num_leaves', list(range(20, 250, 10))),
    
    # specifies the minimum samples per leaf node.
    # the minimum number of samples (data) to group into a leaf. 
    # The parameter can greatly assist with overfitting: larger sample
    # sizes per leaf will reduce overfitting (but may lead to under-fitting).
    'min_child_samples': hp.choice('min_child_samples', list(range(100, 250, 10))),
    
    # subsample: represents a fraction of the rows (observations) to be 
    # considered when building each subtree. Tianqi Chen and Carlos Guestrin
    # in their paper A Scalable Tree Boosting System recommend 
    'subsample': hp.choice('subsample', [0.2, 0.4, 0.5, 0.6, 0.7, .8, .9]),
    
    # randomly select a fraction of the features.
    # feature_fraction: controls the subsampling of features used
    # for training (as opposed to subsampling the actual training data in 
    # the case of bagging). Smaller fractions reduce overfitting.
    'feature_fraction': hp.uniform('feature_fraction', 0.4, .8),
    
    # randomly bag or subsample training data.
    'bagging_fraction': hp.uniform('bagging_fraction', 0.4, .9)
    
    # bagging_fraction and bagging_freq: enables bagging (subsampling) 
    # of the training data. Both values need to be set for bagging to be used.
    # The frequency controls how often (iteration) bagging is used. Smaller
    # fractions and frequencies reduce overfitting.
}

In [57]:


# Set algoritm parameters
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)

# Print best parameters
best_params = space_eval(space, best)




############## New Run ################
params = {'max_depth': 14, 'gamma': '0.148', 'subsample': '0.80', 'reg_alpha': '0.340', 'reg_lambda': '0.174', 'learning_rate': '0.138', 'num_leaves': '70.000', 'colsample_bytree': '0.419', 'min_child_samples': '170.000', 'feature_fraction': '0.592', 'bagging_fraction': '0.776'}
1 CV - score: 0.8998
2 CV - score: 0.8994
3 CV - score: 0.918
4 CV - score: 0.8914
5 CV - score: 0.9266
6 CV - score: 0.9189
7 CV - score: 0.9189
Total Time Run: 3.51
Mean ROC_AUC: 0.9104211024698647

############## New Run ################
params = {'max_depth': 11, 'gamma': '0.320', 'subsample': '0.60', 'reg_alpha': '0.171', 'reg_lambda': '0.382', 'learning_rate': '0.013', 'num_leaves': '30.000', 'colsample_bytree': '0.772', 'min_child_samples': '150.000', 'feature_fraction': '0.575', 'bagging_fraction': '0.858'}
1 CV - score: 0.8985
2 CV - score: 0.9125
3 CV - score: 0.9249
4 CV - score: 0.9039
5 CV - score: 0.9307
6 CV - score: 0.9265
7 CV - score: 0.9211
Total Time 

In [58]:
print("BEST PARAMS: ", best_params)

best_params['max_depth'] = int(best_params['max_depth'])

BEST PARAMS:  {'bagging_fraction': 0.8787058795253496, 'colsample_bytree': 0.5427642005224291, 'feature_fraction': 0.7435023738832408, 'gamma': 0.5030791234636088, 'learning_rate': 0.06681428345787745, 'max_depth': 17.0, 'min_child_samples': 100, 'num_leaves': 230, 'reg_alpha': 0.23355695093321022, 'reg_lambda': 0.18155241402663785, 'subsample': 0.9}


In [0]:
clf = xgb.XGBClassifier(
    n_estimators=300,
    **best_params,
    tree_method='gpu_hist'
)

clf.fit(X_train, y_train)

y_preds = clf.predict_proba(X_test)[:,1] 

In [0]:
sample_submission['isFraud'] = y_preds
sample_submission.to_csv('XGB_hypopt_model.csv')